!pip install uproot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import uproot
import pandas as pd
%matplotlib inline

# Generate toy data

New Physics model 'NP1' is Gaussian (0,1), SM is Gaussian (0.5,1)

In [ ]:
nEvents = 10**6
gNP1 = (0,1)
gSM = (0.5,1)

X = np.concatenate([np.random.normal(loc=gNP1[0],scale=gNP1[1], size=nEvents//2), 
     np.random.normal(loc=gSM[0],scale=gSM[1], size=nEvents//2)])
Y = np.concatenate([np.ones(nEvents//2), np.zeros(nEvents//2)])

plt.hist(X[Y==0], bins=50, histtype="step", range=(-4,5), label="SM")
plt.hist(X[Y==1], bins=50, histtype="step", range=(-4,5), label="NP1")
plt.legend()
plt.show()

### Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X.reshape(-1,1) ,Y, train_size=nEvents//2, shuffle=True)

# Define likelihood ratio estimators

$$LR = \frac{P(data| NP_1)}{P(data| SM)}$$

In [ ]:
# Since the data is Gaussian, we know the analytical form of the likelihood ratio
class trueLR:
    def predict(self, X):
        from scipy.stats import multivariate_normal
        #Evaluate the probability density function at point X
        rv_NP1 = multivariate_normal(??, ??).pdf
        rv_SM = ??
        LR = rv_NP1(X)/rv_SM(X)                                                        
        return LR
trueLR_model = trueLR()

In [ ]:
LR_SM_test = trueLR_model.predict(X=X_test[Y_test==0])
LR_NP1_test = trueLR_model.predict(X=X_test[Y_test==1])

plt.hist(LR_SM_test,
         bins=100, histtype="step", range=(0,10), label="LR SM Events")
plt.hist(LR_NP1_test,
         bins=100, histtype="step", range=(0,10), label="LR NP1 Events")
plt.legend()
plt.show()

In [ ]:
print ("Combined LR SM: ",LR_SM_test.prod(), "Combined LR NP: ", LR_NP1_test.prod())
print ("Rreducing stats to only 100 events")
print ("Combined LR back: ",LR_SM_test[:100].prod(), "Combined LR signal: ", LR_NP1_test[:100].prod())

# Assume you expect to observe only 10 events in data

In [ ]:
nData = 10
nPseudoExp = 10**4
dataset_LR_SM = LR_SM_test[:nPseudoExp*nData].reshape(nData,-1).prod(axis=0)
dataset_LR_NP1 = LR_NP1_test[:nPseudoExp*nData].reshape(nData,-1).prod(axis=0)

plt.hist(dataset_LR_SM,
         bins=100, histtype="step", range=(0,10), label="True LR: pseudo-experiments with SM")
plt.hist(dataset_LR_NP1,
         bins=100, histtype="step", range=(0,10), label="True LR: pseudo-experiments with NP1")
threshold = 1.35
plt.axvline(x = threshold, color = 'r', label = 'Threshold to reject SM')
plt.legend()
plt.show()

In [ ]:
(dataset_LR_SM > threshold).sum() / dataset_LR_SM.shape[0], (dataset_LR_NP1 > threshold).sum() / dataset_LR_NP1.shape[0]

# How to get this in a "Likelihood-Free" way ?

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(X_train.shape[1],))
D = Dense(256, activation="relu")(inputs) # first layer
D = Dense(256, activation="relu")(D) 
D = Dense(1, activation="sigmoid")(D) # last layer
clf = Model(inputs=inputs, outputs=D)

clf.compile(loss="binary_crossentropy", optimizer="adam")

In [ ]:
clf.fit(x=X_train, y=Y_train, epochs=10, batch_size=1024)

$$ LR = \frac{s}{1-s} $$

In [ ]:
y_pred = 

In [ ]:
lr_pred = 

In [ ]:
LR_pred_SM_test = lr_pred[Y_test==0]
LR_pred_NP1_test = lr_pred[Y_test==1]

Let's see how close the estimated LRs are to the truth

In [ ]:
print("SM data")
print ("True LR: ", LR_SM_test[:5], "\nPred LR: ", LR_pred_SM_test[:5])
print("NP1 data")
print ("True LR: ", LR_NP1_test[:5], "\nPred LR: ", LR_pred_NP1_test[:5])


# Uses: 
- **Hypothesis testing**
- Negative weighted events: reweight MC with negative weights into MC with only positive weights
- Re-weighing Pythia to Herwig etc
- Unfolding

In [ ]:
# Same as before, but now the estimated LR coems from the classifier


But is this estimate good enough? Is it accurate everywhere? 
What if we have a bias in some phase space? 

Need to visualise the LR leant, so flip the classifier into a 'generative model'

## Diagnostics: Re-weighting SM events to look like NP1

In [ ]:
weights = lr_pred

plt.hist(X_test[Y_test==0], bins=50, histtype="step", density=1, range=(-4,5), label="SM")
plt.hist(X_test[Y_test==1], bins=50, histtype="step", density=1, range=(-4,5), label="NP1")
plt.hist(X_test[Y_test==0], weights=weights[Y_test==0], bins=50, histtype="step", density=1, range=(-4,5), label="Reweight SM -> NP1")
plt.legend()
plt.xlabel("Observable 1")
plt.ylabel("Arbitrary units")
plt.title("Diagnostics with reweighting")
#plt.savefig("reweight.pdf")
plt.show()

## Diagnostics: Calibration curve

In [ ]:
# predictions on training set
y_pred_train = clf.predict(X_train, batch_size=4*1024).ravel()

In [ ]:
from extra_functions import compare_train_test

compare_train_test(
    y_pred_train,
    Y_train,
    y_pred,
    Y_test,
    xlabel="Classifier Score",
    title="Score Distribution",
)

### Now for calibration curve, we scan over different thresholds and plot the purity. 
Purity of NP1 events for a given threshold cut is nNP1/(nNP1+nSM) that pass the cut.

In [ ]:
from sklearn.calibration import calibration_curve
prob_true, prob_pred = calibration_curve(Y_test, y_pred, n_bins=50)
plt.plot([0, 1], [0, 1], linestyle='--', label='ideal')
plt.plot(prob_true, prob_pred, marker='.', label='clf test')
# prob_true, prob_pred = calibration_curve(Y_train, y_pred_train, n_bins=50)
# plt.plot(prob_true, prob_pred, marker='.', label='clf train')
plt.legend()
plt.title("Calibration curve")
plt.xlabel("True purity")
plt.ylabel("Predicted purity")
plt.show()

## Exercise:
- Try plots with a poorly trained classifier

## Optional:
- Try 2D Gaussian models for SM and NP1

# Now on physics dataset

We will assume that B is H_0 and S is H_1 (even though actually H1 as S+B makes more sense)

In [ ]:
# same data & data loading code as S vs B tutorial
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

filename = "dataFiles/dataWW_d1.root"
file = uproot.open(filename)

# show what is inside the root file loaded from uproot
print(file.classnames())
print(file.keys())

tree = file["tree_event"]  # select the TTree inside the root file
dfall = tree.arrays(library="pd")  # convert uproot TTree into pandas dataframe

label_weights = (dfall[dfall.label == 0].mcWeight.sum(), dfall[dfall.label == 1].mcWeight.sum())
fulldata = dfall[
    #(dfall.lep_n == 2) & (dfall.mcWeight > 0)
    (dfall.lep_n == 2)
]  # only keep events with exactly two leptons

target = fulldata["label"]
weights = fulldata["mcWeight"]

#true_class_weights = [weights[target == 0].sum(), weights[target == 1].sum()]

featureList = ['met_et', 'met_phi', 'lep_n',
       'lep_pt_0', 'lep_pt_1', 'lep_eta_0', 'lep_eta_1', 'lep_phi_0',
       'lep_phi_1', 'lep_E_0', 'lep_E_1', 'lep_charge_0', 'lep_charge_1',
       'lep_type_0', 'lep_type_1', 'jet_n', 'jet_pt_0', 'jet_pt_1',
       'jet_eta_0', 'jet_eta_1', 'jet_phi_0', 'jet_phi_1', 'jet_E_0',
       'jet_E_1']
data = pd.DataFrame(fulldata, columns=features)


In [ ]:
train_size = 0.75  # fraction of sample used for training

XP_train, XP_test, yP_train, yP_test, weightsP_train, weightsP_test = train_test_split(
    data, target, weights, train_size=train_size)

In [ ]:
prec = 2

scalerP = StandardScaler()
XP_train = scalerP.fit_transform(XP_train)
XP_test = scalerP.transform(XP_test)

class_weights_train = (weightsP_train[yP_train == 0].sum(), weightsP_train[yP_train == 1].sum())

for i in range(len(class_weights_train)):
    weightsP_train[yP_train == i] *= (
        max(class_weights_train) / class_weights_train[i]
    )  # equalize number of background and signal event

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(XP_train.shape[1],))
C = Dense(1024, activation="relu")(inputs) # first layer
C = Dense(1024, activation="relu")(C) 
C = Dense(1, activation="sigmoid")(C) # last layer
model = Model(inputs=inputs, outputs=C)

model.compile(loss="binary_crossentropy", optimizer="adam")

In [ ]:
model.fit(x=XP_train, y=yP_train, epochs=10, batch_size=1024)

In [ ]:
yP_pred = model.predict(XP_test, batch_size=4*1024).ravel()
yP_train_pred = model.predict(XP_train, batch_size=4*1024).ravel()
lrP_pred = yP_pred/(1-yP_pred)

## Typical plots

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# easier to remove negative weighted events for fast AUC calucaltion
posWeightsTrain, posWeightsTest = weightsP_train>0, weightsP_test>0

fpr_test, tpr_test, _ = roc_curve(y_true=yP_test[posWeightsTest],
                                  y_score=yP_pred[posWeightsTest], 
                                  sample_weight=weightsP_test[posWeightsTest])
fpr_train, tpr_train, _ = roc_curve(y_true=yP_train[posWeightsTrain],
                                    y_score=yP_train_pred[posWeightsTrain], 
                                    sample_weight=weightsP_train[posWeightsTrain])
auc_test = roc_auc_score(y_true=yP_test[posWeightsTest], 
                         y_score=yP_pred[posWeightsTest],
                         sample_weight=weightsP_test[posWeightsTest])
auc_train = roc_auc_score(y_true=yP_train[posWeightsTrain],
                          y_score=yP_train_pred[posWeightsTrain],
                          sample_weight=weightsP_train[posWeightsTrain])
plt.plot(fpr_test, tpr_test, color='tab:blue', lw=2, ls="--", label=f"Test set auc: {auc_test:.4f}")
plt.plot(fpr_train, tpr_train, color='tab:orange', lw=2, ls=":", label=f"Training set auc: {auc_train:.4f}")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from extra_functions import compare_train_test

compare_train_test(
    yP_train_pred,
    yP_train,
    yP_pred,
    yP_test,
    xlabel="NN Score",
    title="Score distribution",
    weights_train=weightsP_train.values,
    weights_test=weightsP_test.values,
)

## Advanced Diagnostics

In [ ]:
reweights = weightsP_test * lrP_pred
f = 3; rangee = (-2,10)  # which observable to plot, what range
plt.hist(XP_test[yP_test==0, f], weights=weightsP_test[yP_test==0], 
         bins=50, range=rangee, histtype="step", density=1, label="B")
plt.hist(XP_test[yP_test==1, f], weights=weightsP_test[yP_test==1], 
         bins=50, range=rangee, histtype="step", density=1, label="S")
plt.hist(XP_test[yP_test==0, f], weights=reweights[yP_test==0], 
         bins=50, range=rangee, histtype="step", density=1, label="Reweight B -> S")
plt.xlabel(featureList[f])
plt.ylabel("Arbitrary units")
plt.title("Diagnostics with reweighting")
plt.legend()
plt.show()

# try plotting f=4, how would you improve it?

In [ ]:
from sklearn.calibration import calibration_curve
prob_true, prob_pred = calibration_curve(yP_test, yP_pred, n_bins=50)
plt.plot([0, 1], [0, 1], linestyle='--', label='ideal')
plt.plot(prob_true, prob_pred, marker='.', label='clf test')
prob_true, prob_pred = calibration_curve(yP_train, yP_train_pred, n_bins=50)
plt.plot(prob_true, prob_pred, marker='.', label='clf train')
plt.legend()
plt.xlabel("True purity")
plt.ylabel("Predicted purity")
plt.show()

## Exercise:
- How can we improve the performance? Preprocessing & HPO, regularization (dropouts), early-stopping
- Ensembling

## Optional:
- Investigate phase space with poor performance

# Omnifold: Multi-dimensional unbinned unfolding
Content borrowed from Vinicius Mikuni (https://github.com/usatlas-ml-training/lbnl-2023/blob/main/unfolding/DIS_Omnifold-sol.ipynb)

In [ ]:
import omnifold as of

In [ ]:
N_Events = 10**5

# Synthetic
theta0_G = np.random.normal(0.2, 0.8, N_Events)  # Generator-level synthetic sample
theta0_S = np.array([(x + np.random.normal(0, 0.5)) for x in theta0_G])  # Detector smearing for synthetic sample

theta0 = np.stack([theta0_G, theta0_S], axis=1)

# Natural
theta_unknown_G = np.random.normal(0, 1, N_Events)
theta_unknown_S = np.array([(x + np.random.normal(0, 0.5)) for x in theta_unknown_G])

In [ ]:
_, _, _ = plt.hist(theta0_G, bins=np.linspace(-3, 3, 20), color='blue', alpha=0.5, label="MC, true")
_, _, _ = plt.hist(theta0_S, bins=np.linspace(-3, 3, 20), histtype="step", color='black', ls=':', label="MC, reco")
_, _, _ = plt.hist(theta_unknown_G, bins=np.linspace(-3, 3, 20), color='orange', alpha=0.5, label="Data, true")
_, _, _ = plt.hist(theta_unknown_S, bins=np.linspace(-3, 3, 20), histtype="step", color='black', label="Data, reco")
plt.xlabel("x")
plt.ylabel("events")
plt.legend(frameon=False)

### Define the Model Architecture

In [ ]:
inputs = Input((1,))
hidden_layer_1 = Dense(50, activation='relu')(inputs)
hidden_layer_2 = Dense(50, activation='relu')(hidden_layer_1)
hidden_layer_3 = Dense(50, activation='relu')(hidden_layer_2)
outputs = Dense(1, activation='sigmoid')(hidden_layer_3)
model_gaus = Model(inputs=inputs, outputs=outputs)

In [ ]:
N_Iterations = 2
myweights = of.omnifold(theta0_G, theta0_S, theta_unknown_S, N_Iterations, model_gaus)

### Plot unfolded distributions

In [ ]:
_, _, _ = plt.hist(theta0_G, bins=np.linspace(-3, 3, 20), color='blue', alpha=0.5, label="MC, true")
_, _, _ = plt.hist(theta_unknown_G, bins=np.linspace(-3, 3, 20), color='orange', alpha=0.5, label="Data, true")
_, _, _ = plt.hist(
    theta0_G,
    weights=myweights[-1, 1, :],
    bins=np.linspace(-3, 3, 20),
    color='black',
    histtype="step",
    label="OmniFolded",
    lw=2,
)
plt.xlabel("x")
plt.ylabel("events")
plt.legend(frameon=False)

### Plot the distribution at reco level instead of gen level

In [ ]:
_,_,_=plt.hist(theta0_S,bins=np.linspace(-3,3,20),color='blue',alpha=0.5,label="MC, reco")
_,_,_=plt.hist(theta_unknown_S,bins=np.linspace(-3,3,20),color='orange',alpha=0.5,label="Data, reco")
_,_,_=plt.hist(theta0_S,weights=myweights[-1, 1, :], bins=np.linspace(-3,3,20),color='black',histtype="step",label="OmniFolded",lw=2)
plt.xlabel("x")
plt.ylabel("events")
plt.legend(frameon=False)